# lightGBM part2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [117]:
np.square(base_train['age'])

0         441
1         841
2         361
3         289
4        2209
         ... 
11895     529
11896     961
11897     441
11898     841
11899     289
Name: age, Length: 11900, dtype: int64

In [138]:
base_train = pd.read_csv('data/train2.csv')
base_test = pd.read_csv('data/test2.csv')

base_train = base_train.drop('index', axis=1)
base_train = base_train.drop('native-country', axis=1)
base_train = base_train.drop('fnlwgt', axis=1)
# base_train = base_train.drop('race', axis=1)

base_test = base_test.drop('index', axis=1)
base_test = base_test.drop('native-country', axis=1)
base_test = base_test.drop('fnlwgt', axis=1)
# base_test = base_test.drop('race', axis=1)


# 特徴量の追加

base_train['age-median'] = base_train['age'] - base_train['age'].median()
# base_train['age-mean'] = base_train['age'] - base_train['age'].mean()
# base_train['age-square'] = np.square(base_train['age'])
base_train['education-num-median'] = base_train['education-num'] - base_train['education-num'].median()

base_test['age-median'] = base_test['age'] - base_train['age'].median()
# base_test['age-mean'] = base_test['age'] - base_train['age'].mean()
# base_test['age-square'] = np.square(base_test['age'])
base_test['education-num-median'] = base_test['education-num'] - base_train['education-num'].median()

# "Y"の順序
cols = base_train.columns.tolist()
cols.remove("Y")
cols.append("Y")
base_train = base_train[cols]

base_train.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,age-median,education-num-median,Y
0,21,3,13,10,0,1,3,2,0,-3.0,0.0,0
1,29,3,10,9,0,1,1,2,0,5.0,-1.0,0
2,19,3,9,13,2,8,1,2,0,-5.0,3.0,0
3,17,3,10,9,2,2,3,2,1,-7.0,-1.0,0
4,47,3,13,10,1,2,0,2,1,23.0,0.0,0


In [139]:
# categorical_features = ["workclass","fnlwgt","education","marital-status","occupation","relationship","race","sex","native-country"]
categorical_features = ["workclass","education","marital-status","occupation","relationship","race","sex"]
cat_columns = []
for i,v in enumerate(base_train.columns):
    if v in categorical_features:
        cat_columns.append(i)
print(cat_columns)

[1, 2, 4, 5, 6, 7, 8]


### ハイパラメータ最適化

In [140]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(base_train.iloc[:,:-1],base_train.iloc[:,-1], random_state=0, test_size=0.2)
X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train, random_state=0, test_size=0.1)

In [141]:
import optuna.integration.lightgbm as optuna_lgb
from sklearn.metrics import accuracy_score

trains = lgb.Dataset(X_train, y_train,categorical_feature=categorical_features)
valids = lgb.Dataset(X_valid, y_valid,categorical_feature=categorical_features)

params = {
    "objective": "binary",
    "metric":"auc",
    "boosting_type": "gbdt",
    "verbosity": -1,
}
model = optuna_lgb.train(params, trains, valid_sets=valids,
                    verbose_eval=False, num_boost_round=1000,early_stopping_rounds=100)

prediction = np.rint(model.predict(X_valid, num_iteration=model.best_iteration))
accuracy = accuracy_score(y_valid, prediction)

best_params = model.params
print("Best params:", best_params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))
    
print(model.best_iteration)
optimum_boost_rounds = model.best_iteration
print("optimum_boost_rounds: {}".format(optimum_boost_rounds))

[I 2021-02-16 23:03:58,213] A new study created in memory with name: no-name-9e434b71-7ee8-4695-b88d-211b79cad1f2



  0%|          | 0/7 [00:00<?, ?it/s]


feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))



feature_fraction, val_score: 0.910591:   0%|          | 0/7 [00:00<?, ?it/s]


feature_fraction, val_score: 0.910591:  14%|#4        | 1/7 [00:00<00:02,  2.85it/s][I 2021-02-16 23:03:58,602] Trial 0 finished w

Best params: {'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 3, 'feature_fraction': 0.8999999999999999, 'bagging_fraction': 0.6493593458472748, 'bagging_freq': 6, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100, 'categorical_column': [1, 2, 4, 5, 6, 7, 8]}
  Accuracy = 0.8476890756302521
  Params: 
    objective: binary
    metric: auc
    boosting_type: gbdt
    verbosity: -1
    feature_pre_filter: False
    lambda_l1: 0.0
    lambda_l2: 0.0
    num_leaves: 3
    feature_fraction: 0.8999999999999999
    bagging_fraction: 0.6493593458472748
    bagging_freq: 6
    min_child_samples: 20
    num_iterations: 1000
    early_stopping_round: 100
    categorical_column: [1, 2, 4, 5, 6, 7, 8]
398
optimum_boost_rounds: 398


In [142]:
import lightgbm as lgb

trains = lgb.Dataset(X_train, y_train,categorical_feature=categorical_features)
valids = lgb.Dataset(X_valid, y_valid,categorical_feature=categorical_features)

best_model = lgb.train(best_params, trains,valid_sets=valids,num_boost_round=optimum_boost_rounds,verbose_eval=False)
y_pred = np.round(best_model.predict(X_test)).astype(int)

from sklearn.metrics import confusion_matrix
#混同行列の作成
matrix6 = confusion_matrix(y_test, y_pred)
print("混同行列:\n{}".format(matrix6))

from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
print("正解率(Accuracy):{:.3f}".format(accuracy_score(y_test, y_pred)))
print("適合率(Precision):{:.3f}".format(precision_score(y_test, y_pred)))
print("再現率(Recall):{:.3f}".format(recall_score(y_test, y_pred)))
print("F1値(Accuracy):{:.3f}".format(f1_score(y_test, y_pred)))

importance = pd.DataFrame(best_model.feature_importance(), index = X_train.columns, columns=["importance"])
display(importance.sort_values("importance", ascending= False))


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


混同行列:
[[1618  157]
 [ 211  394]]
正解率(Accuracy):0.845
適合率(Precision):0.715
再現率(Recall):0.651
F1値(Accuracy):0.682


,importance
occupation,198
age,176
education,135
workclass,64
education-num,63
marital-status,51
relationship,41
race,29
age-median,22
sex,12


### アンサンブル

In [143]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split


output_result = pd.DataFrame()
for _ in range(20):
    X_train,X_valid,y_train,y_valid = train_test_split(base_train.iloc[:,:-1],base_train.iloc[:,-1], random_state=_, test_size=0.2)
    
    trains = lgb.Dataset(X_train, y_train,categorical_feature=categorical_features)
    valids = lgb.Dataset(X_valid, y_valid,categorical_feature=categorical_features)
    
    model = lgb.train(best_params, trains,valid_sets=valids,num_boost_round=optimum_boost_rounds,verbose_eval=False)
    pred = np.round(best_model.predict(base_test)).astype(int)
    
    output = pd.DataFrame(pred,columns=['pred' + str(_+1)])
    #各予測結果を格納
    if _ == 0:
        output_result = output
    else:
        output_result = pd.concat([output_result,output],axis=1)
        
output_result.head()

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,pred9,pred10,pred11,pred12,pred13,pred14,pred15,pred16,pred17,pred18,pred19,pred20
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [145]:
df_mean = np.round(output_result.mean(axis='columns')).astype(int)

_test = pd.read_csv('data/test2.csv')
df_result = pd.concat([_test['index'],df_mean],axis=1)

df_result.to_csv('data/submit2.csv',index=None,header=None)

In [144]:
output_result.describe()

,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,pred9,pred10,pred11,pred12,pred13,pred14,pred15,pred16,pred17,pred18,pred19,pred20
count,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000,5100.000000
mean,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314,0.234314
std,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611,0.423611
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
